# Import Libraries

In [1]:
import pandas as pd
import numpy as np

# Obtain Data

Read csv with player stats into pandas

In [2]:
stats_df = pd.read_csv('Data/player_stats.csv')
print('stats_df shape:', stats_df.shape)
stats_df.head()

stats_df shape: (530, 29)


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Álex Abrines,SG,25,OKC,31,2,588,56,157,0.357,...,0.923,5,43,48,20,17,6,14,53,165
1,Quincy Acy,PF,28,PHO,10,0,123,4,18,0.222,...,0.700,3,22,25,8,1,4,4,24,17
2,Jaylen Adams,PG,22,ATL,34,1,428,38,110,0.345,...,0.778,11,49,60,65,14,5,28,45,108
3,Steven Adams,C,25,OKC,80,80,2669,481,809,0.595,...,0.500,391,369,760,124,117,76,135,204,1108
4,Bam Adebayo,C,21,MIA,82,28,1913,280,486,0.576,...,0.735,165,432,597,184,71,65,121,203,729


Read csv with combine data into pandas

In [3]:
combine_df = pd.read_csv('Data/combine_measurements.csv')
print('combine_df shape:', combine_df.shape)
combine_df.head()

combine_df shape: (1273, 11)


,Year,Player,Pos,Body Fat %,Hand Length (Inches),Hand Width (Inches),Height W/O Shoes,Height W/ Shoes,Standing Reach,Weight (Lbs),Wingspan
0,2000-01,Primoz Brezec,C,-%,-,-,7' 0.75'',NaN,9' 2'',243.0,7' 2''
1,2000-01,Ndongo N'diaye,C,-%,-,-,6' 11.5'',NaN,9' 4.5'',208.0,7' 3.5''
2,2000-01,Karim Shabazz,C,-%,-,-,6' 11.5'',NaN,9' 3.5'',224.0,7' 3''
3,2000-01,Dan McClintock,C-PF,-%,-,-,6' 11'',NaN,8' 11'',250.0,7' 1.5''
4,2000-01,Ernest Brown,C,-%,-,-,6' 10'',NaN,9' 1.5'',222.5,7' 1.25''


Merge combine_df and stats_df together

In [4]:
df = pd.merge(stats_df, combine_df, on='Player')
print('df shape:', df.shape)
df.head()

df shape: (325, 39)


,Player,Pos_x,Age,Tm,G,GS,MP,FG,FGA,FG%,...,Year,Pos_y,Body Fat %,Hand Length (Inches),Hand Width (Inches),Height W/O Shoes,Height W/ Shoes,Standing Reach,Weight (Lbs),Wingspan
0,Quincy Acy,PF,28,PHO,10,0,123,4,18,0.222,...,2012-13,PF,7.10%,9.50,11.00,6' 6.5'',6' 7.75'',8' 10.5'',223.8,7' 2.75''
1,Steven Adams,C,25,OKC,80,80,2669,481,809,0.595,...,2013-14,C,6.65%,9.50,11.00,6' 10.75'',7' 0'',9' 1.5'',254.5,7' 4.5''
2,LaMarcus Aldridge,C,33,SAS,81,81,2687,684,1319,0.519,...,2006-07,PF-C,8.70%,-,-,6' 10'',6' 11.25'',9' 2'',234.0,7' 4.75''
3,Rawle Alkins,SG,21,CHI,10,1,120,13,39,0.333,...,2017-18,SG-SF,11.00%,8.75,10.00,6' 2.5'',6' 3.75'',8' 3'',222.6,6' 8.75''
4,Rawle Alkins,SG,21,CHI,10,1,120,13,39,0.333,...,2018-19,SG,8.90%,8.50,10.00,6' 2.75'',6' 4.25'',8' 3'',217.4,6' 8.75''


# Scrub Data

Use df.info() to determine what scrubbing needed

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325 entries, 0 to 324
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Player                325 non-null    object 
 1   Pos_x                 325 non-null    object 
 2   Age                   325 non-null    int64  
 3   Tm                    325 non-null    object 
 4   G                     325 non-null    int64  
 5   GS                    325 non-null    int64  
 6   MP                    325 non-null    int64  
 7   FG                    325 non-null    int64  
 8   FGA                   325 non-null    int64  
 9   FG%                   322 non-null    float64
 10  3P                    325 non-null    int64  
 11  3PA                   325 non-null    int64  
 12  3P%                   305 non-null    float64
 13  2P                    325 non-null    int64  
 14  2PA                   325 non-null    int64  
 15  2P%                   3

Drop columns not useful for modeling

In [6]:
drop_columns = ['Player', 'Pos_x', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA','FG%', '3P', '3PA', '2P', '2PA', '2P%',
                'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 
                'Pos_y']
df.drop(columns=drop_columns, inplace=True)
print('df shape:', df.shape)
df.head()

df shape: (325, 9)


,3P%,Body Fat %,Hand Length (Inches),Hand Width (Inches),Height W/O Shoes,Height W/ Shoes,Standing Reach,Weight (Lbs),Wingspan
0,0.133,7.10%,9.50,11.00,6' 6.5'',6' 7.75'',8' 10.5'',223.8,7' 2.75''
1,0.000,6.65%,9.50,11.00,6' 10.75'',7' 0'',9' 1.5'',254.5,7' 4.5''
2,0.238,8.70%,-,-,6' 10'',6' 11.25'',9' 2'',234.0,7' 4.75''
3,0.250,11.00%,8.75,10.00,6' 2.5'',6' 3.75'',8' 3'',222.6,6' 8.75''
4,0.250,8.90%,8.50,10.00,6' 2.75'',6' 4.25'',8' 3'',217.4,6' 8.75''


Replace all '-' datapoints indicating a missing value in df with np.nan

In [7]:
df.replace({'-': np.nan, '-%': np.nan}, inplace=True)
df.head()

,3P%,Body Fat %,Hand Length (Inches),Hand Width (Inches),Height W/O Shoes,Height W/ Shoes,Standing Reach,Weight (Lbs),Wingspan
0,0.133,7.10%,9.50,11.00,6' 6.5'',6' 7.75'',8' 10.5'',223.8,7' 2.75''
1,0.000,6.65%,9.50,11.00,6' 10.75'',7' 0'',9' 1.5'',254.5,7' 4.5''
2,0.238,8.70%,NaN,NaN,6' 10'',6' 11.25'',9' 2'',234.0,7' 4.75''
3,0.250,11.00%,8.75,10.00,6' 2.5'',6' 3.75'',8' 3'',222.6,6' 8.75''
4,0.250,8.90%,8.50,10.00,6' 2.75'',6' 4.25'',8' 3'',217.4,6' 8.75''


Update 'Body Fat %' column data type from string to float

In [8]:
def scrub_bf(val):
    if type(val) == float:
        return val
    else:
        return float(val.replace('%', ''))/100
    
df['Body Fat %'] = df['Body Fat %'].apply(scrub_bf)
df.head()

,3P%,Body Fat %,Hand Length (Inches),Hand Width (Inches),Height W/O Shoes,Height W/ Shoes,Standing Reach,Weight (Lbs),Wingspan
0,0.133,0.0710,9.50,11.00,6' 6.5'',6' 7.75'',8' 10.5'',223.8,7' 2.75''
1,0.000,0.0665,9.50,11.00,6' 10.75'',7' 0'',9' 1.5'',254.5,7' 4.5''
2,0.238,0.0870,NaN,NaN,6' 10'',6' 11.25'',9' 2'',234.0,7' 4.75''
3,0.250,0.1100,8.75,10.00,6' 2.5'',6' 3.75'',8' 3'',222.6,6' 8.75''
4,0.250,0.0890,8.50,10.00,6' 2.75'',6' 4.25'',8' 3'',217.4,6' 8.75''


Update 'Hand Length (Inches)', 'Hand Width (Inches)' and 'Weight (Lbs)' column data types to float

In [9]:
dtype_dict = {'Hand Length (Inches)':float, 'Hand Width (Inches)': float, 'Weight (Lbs)': float}
df = df.astype(dtype_dict)
df.head()

,3P%,Body Fat %,Hand Length (Inches),Hand Width (Inches),Height W/O Shoes,Height W/ Shoes,Standing Reach,Weight (Lbs),Wingspan
0,0.133,0.0710,9.50,11.0,6' 6.5'',6' 7.75'',8' 10.5'',223.8,7' 2.75''
1,0.000,0.0665,9.50,11.0,6' 10.75'',7' 0'',9' 1.5'',254.5,7' 4.5''
2,0.238,0.0870,NaN,NaN,6' 10'',6' 11.25'',9' 2'',234.0,7' 4.75''
3,0.250,0.1100,8.75,10.0,6' 2.5'',6' 3.75'',8' 3'',222.6,6' 8.75''
4,0.250,0.0890,8.50,10.0,6' 2.75'',6' 4.25'',8' 3'',217.4,6' 8.75''


Update remaining object type columns measured in feet to float type measured in inches

In [10]:
def feet_to_inches(val):
    if type(val) == float:
        return val
    else:
        val = val.replace("'", "")
        val = val.replace('"', '')
        return 12*float(val.split(' ')[0]) + float(val.split(' ')[1])

feet_cols = ['Height W/O Shoes', 'Height W/ Shoes', 'Standing Reach', 'Wingspan']
for col in feet_cols:
    df[col] = df[col].apply(feet_to_inches)
df.head()

,3P%,Body Fat %,Hand Length (Inches),Hand Width (Inches),Height W/O Shoes,Height W/ Shoes,Standing Reach,Weight (Lbs),Wingspan
0,0.133,0.0710,9.50,11.0,78.50,79.75,106.5,223.8,86.75
1,0.000,0.0665,9.50,11.0,82.75,84.00,109.5,254.5,88.50
2,0.238,0.0870,NaN,NaN,82.00,83.25,110.0,234.0,88.75
3,0.250,0.1100,8.75,10.0,74.50,75.75,99.0,222.6,80.75
4,0.250,0.0890,8.50,10.0,74.75,76.25,99.0,217.4,80.75


Confirm all columns are now float type

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325 entries, 0 to 324
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   3P%                   305 non-null    float64
 1   Body Fat %            312 non-null    float64
 2   Hand Length (Inches)  258 non-null    float64
 3   Hand Width (Inches)   258 non-null    float64
 4   Height W/O Shoes      323 non-null    float64
 5   Height W/ Shoes       316 non-null    float64
 6   Standing Reach        323 non-null    float64
 7   Weight (Lbs)          323 non-null    float64
 8   Wingspan              324 non-null    float64
dtypes: float64(9)
memory usage: 25.4 KB
